# Rossmann Store Sales

# Iteration 3

# Bibliotheken importieren



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set(rc={'figure.figsize':(20,10)})

C:\tools\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\tools\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\tools\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


## Daten importieren

In [2]:
import pandas as pd
from pandas.api.types import CategoricalDtype
# pd.set_option('display.max_columns', None)

dtypes_train = {
    'Store' : 'int',
    'DayOfWeek' : 'category',
    'Sales' : 'int',
    'Customers' : 'int',
    'Open' : 'bool',
    'Promo' : 'bool',
    'StateHoliday' : 'category',
    'SchoolHoliday' : 'bool'
}
train = pd.read_csv('train.csv', parse_dates = ['Date'], dtype = dtypes_train)
# display(train.info())
# display(train.head())

dtypes_store = {
    'Store' : 'int',
    'StoreType' : 'category',
    'Assortment' : 'category',
    'CompetitionDistance' : 'object',
    'CompetitionOpenSinceMonth' : 'object',
    'CompetitionOpenSinceYear' : 'object',
    'Promo2' : 'bool',
    'Promo2SinceWeek' : 'object',
    'Promo2SinceYear' : 'object',
    'PromoInterval' : 'object'
}

store = pd.read_csv('store.csv', dtype = dtypes_store)
# display(store.info())
# display(store.head())

demand = train.merge(store, how = 'inner', on = 'Store')
demand.set_index(['Date'], inplace = True)
demand.sort_index(axis = 0, ascending = True, inplace = True)
display(demand.info())
display(demand.head())
# display(daily_demand.info())
# display(daily_demand.head())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   Store                      1017209 non-null  int32   
 1   DayOfWeek                  1017209 non-null  category
 2   Sales                      1017209 non-null  int32   
 3   Customers                  1017209 non-null  int32   
 4   Open                       1017209 non-null  bool    
 5   Promo                      1017209 non-null  bool    
 6   StateHoliday               1017209 non-null  category
 7   SchoolHoliday              1017209 non-null  bool    
 8   StoreType                  1017209 non-null  category
 9   Assortment                 1017209 non-null  category
 10  CompetitionDistance        1014567 non-null  object  
 11  CompetitionOpenSinceMonth  693861 non-null   object  
 12  CompetitionOpenSinceYear   693861 non-nul

None

,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
Date,,,,,,,,,,,,,,,,,
2013-01-01,1115,2,0,0,False,False,a,True,d,c,5350,NaN,NaN,True,22,2012,"Mar,Jun,Sept,Dec"
2013-01-01,746,2,0,0,False,False,a,True,d,c,4330,2,2011,True,35,2011,"Mar,Jun,Sept,Dec"
2013-01-01,171,2,0,0,False,False,a,True,a,a,2640,NaN,NaN,False,NaN,NaN,NaN
2013-01-01,694,2,0,0,False,False,a,True,a,c,460,11,2012,True,40,2014,"Jan,Apr,Jul,Oct"
2013-01-01,396,2,0,0,False,False,a,True,a,c,23130,NaN,NaN,False,NaN,NaN,NaN


# Daten vorbereiten

In [3]:
demand['StateHoliday'] = demand['StateHoliday'].replace(to_replace = ['0','a','b','c'], value = ['None', 'Public holiday', 'Easter', 'Christmas']).astype('category')
demand['Assortment'] = demand['Assortment'].replace(to_replace = ['a','b','c'], value = ['basic', 'extra', 'extended']).astype('category')
demand['DayOfWeek'] = demand['DayOfWeek'].replace(to_replace = ['1','2','3','4','5','6','7'], value = ['Monday', 'Tuesday', 'Wednesday','Thursday','Friday','Saturday','Sunday']).astype('category')

display(demand.info())

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1017209 entries, 2013-01-01 to 2015-07-31
Data columns (total 17 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   Store                      1017209 non-null  int32   
 1   DayOfWeek                  1017209 non-null  category
 2   Sales                      1017209 non-null  int32   
 3   Customers                  1017209 non-null  int32   
 4   Open                       1017209 non-null  bool    
 5   Promo                      1017209 non-null  bool    
 6   StateHoliday               1017209 non-null  category
 7   SchoolHoliday              1017209 non-null  bool    
 8   StoreType                  1017209 non-null  category
 9   Assortment                 1017209 non-null  category
 10  CompetitionDistance        1014567 non-null  object  
 11  CompetitionOpenSinceMonth  693861 non-null   object  
 12  CompetitionOpenSinceYear   693861 non-nul

None

## Visualisieren & Aufbereiten

### Kodierungen

In [4]:
import datetime as dt

data = demand[demand.Store == 1].copy()

# bool'sche Variablen
for col in ['Open', 'SchoolHoliday', 'Promo']:
    data[col] = data[col].replace(to_replace = [True, False], value = [1,0]).astype('int')

# kategorische Variablen
data = pd.get_dummies(data, columns = ['DayOfWeek', 'StateHoliday'], drop_first = False)

# Trend
data['Trend'] = data.index.map(dt.datetime.toordinal)

x = data.drop(columns = 
              [
               'Sales',
               'Store',
               'Customers',
               'StoreType',
               'Assortment',
               'CompetitionDistance',
               'CompetitionOpenSinceMonth',
               'CompetitionOpenSinceYear',
               'Promo2',
               'Promo2SinceWeek',
               'Promo2SinceYear',
               'PromoInterval'
              ]
)

y = data[['Sales']]

display(x)
display(y)

,Open,Promo,SchoolHoliday,DayOfWeek_Friday,DayOfWeek_Monday,DayOfWeek_Saturday,DayOfWeek_Sunday,DayOfWeek_Thursday,DayOfWeek_Tuesday,DayOfWeek_Wednesday,StateHoliday_Christmas,StateHoliday_Easter,StateHoliday_None,StateHoliday_Public holiday,Trend
Date,,,,,,,,,,,,,,,
2013-01-01,0,0,1,0,0,0,0,0,1,0,0,0,0,1,734869
2013-01-02,1,0,1,0,0,0,0,0,0,1,0,0,1,0,734870
2013-01-03,1,0,1,0,0,0,0,1,0,0,0,0,1,0,734871
2013-01-04,1,0,1,1,0,0,0,0,0,0,0,0,1,0,734872
2013-01-05,1,0,1,0,0,1,0,0,0,0,0,0,1,0,734873
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2015-07-27,1,1,1,0,1,0,0,0,0,0,0,0,1,0,735806
2015-07-28,1,1,1,0,0,0,0,0,1,0,0,0,1,0,735807
2015-07-29,1,1,1,0,0,0,0,0,0,1,0,0,1,0,735808


,Sales
Date,
2013-01-01,0
2013-01-02,5530
2013-01-03,4327
2013-01-04,4486
2013-01-05,4997
...,...
2015-07-27,6102
2015-07-28,5011
2015-07-29,4782


###  Merkmalsauswahl

#### Kodierung weiterer Merkmale

In [5]:
# Monat
data['Month'] = data.index.month_name()
data = pd.get_dummies(data, columns = ['Month'], drop_first = False)

# Tag im Monat
# data['DayOfMonth'] = data.index.day
# data = pd.get_dummies(data, columns = ['DayOfMonth'], drop_first = False)

# Woche im Jahr
# data['WeekOfYear'] = data.index.isocalendar().week
# data = pd.get_dummies(data, columns = ['WeekOfYear'], drop_first = False)

x = data.drop(columns = 
              [
               'Sales',
               'Store',
               'Customers',
               'StoreType',
               'Assortment',
               'CompetitionDistance',
               'CompetitionOpenSinceMonth',
               'CompetitionOpenSinceYear',
               'Promo2',
               'Promo2SinceWeek',
               'Promo2SinceYear',
               'PromoInterval'
              ]
)

y = data[['Sales']]

#### Evaluation

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_absolute_error as MAE

k = np.floor(len(data)/28).astype(int)
cv = KFold(n_splits = k)

forecast = RandomForestRegressor(max_depth = 7)

scores = pd.DataFrame(
    cross_validate(
        forecast,
        x,
        y.values.ravel(),
        scoring = 'neg_mean_absolute_error',
        cv = cv,
        verbose = 1,
    )
)

print('MAE mean:\t %4.2f' % (-scores['test_score'].mean()))
print('MAE mad:\t %4.2f' % (scores['test_score'].mad()))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


MAE mean:	 439.67
MAE mad:	 154.45


[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    5.0s finished


#### Beschreibung Merkmalsbedeutung

In [7]:
forecast.fit(x, y.values.ravel())
importance = pd.DataFrame(data = forecast.feature_importances_, index = x.columns)
importance.columns = ['Importance']
display(importance.sort_values(by='Importance', ascending = False))
display(x.shape)

,Importance
Open,0.827263
Promo,0.051608
Month_December,0.036844
Trend,0.030503
DayOfWeek_Saturday,0.021007
DayOfWeek_Monday,0.013024
SchoolHoliday,0.004674
Month_November,0.003260
DayOfWeek_Tuesday,0.002664
DayOfWeek_Friday,0.001949


(942, 27)

#### Automatisierte Merkmalsauswahl

In [11]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import KFold

k = np.floor(len(data)/28).astype(int)
cv = KFold(n_splits = k)

forecast = RandomForestRegressor(max_depth = 7)

selector = RFECV(
    forecast,
    step = 1,
    cv = cv,
    scoring = 'neg_mean_absolute_error'
)

selector = selector.fit(x, y.values.ravel())

selection = pd.DataFrame(data = selector.support_, index = x.columns)
selection.columns = ['Selected']
selection.index.name = 'Variable'
display(selection)

scores = pd.DataFrame(
    cross_validate(
        forecast,
        selector.transform(x),
        y.values.ravel(),
        scoring = 'neg_mean_absolute_error',
        cv = cv,
        verbose = 1
    )
)

print('MAE mean:\t %4.2f' % (-scores['test_score'].mean()))
print('MAE mad:\t %4.2f' % (scores['test_score'].mad()))

,Selected
Variable,
Open,True
Promo,True
SchoolHoliday,True
DayOfWeek_Friday,True
DayOfWeek_Monday,True
DayOfWeek_Saturday,True
DayOfWeek_Sunday,False
DayOfWeek_Thursday,True
DayOfWeek_Tuesday,True


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


MAE mean:	 438.12
MAE mad:	 153.92


[Parallel(n_jobs=1)]: Done  33 out of  33 | elapsed:    6.4s finished


#### Verknüpfung über Modell-Pipeline

In [ ]:
from sklearn.pipeline import Pipeline

k = np.floor(len(data)/28).astype(int)
cv = KFold(n_splits = k)

forecast = RandomForestRegressor(max_depth = 7)

selector = RFECV(
    forecast,
    step = 1,
    cv = cv,
    scoring = 'neg_mean_absolute_error'
)

pipeline = Pipeline([
    ('selector', selector),
    ('forecaster', forecast)
])

scores = pd.DataFrame(
    cross_validate(
        pipeline,
        x,
        y.values.ravel(),
        scoring = 'neg_mean_absolute_error',
        cv = cv,
        verbose = 1,
    )
)

print('MAE mean:\t %4.2f' % (-scores['test_score'].mean()))
print('MAE mad:\t %4.2f' % (scores['test_score'].mad()))

## Modellieren & Schätzen
### Dynamische Regressionsmodelle
#### Prophet

In [ ]:
#!pip install prophet
from prophet import Prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.metrics import mean_absolute_error as MAE

train, test = temporal_train_test_split(demand[demand.Store ==1], test_size = 28)
# train.index.freq = 'D'
# test.index.freq = 'D'

data = pd.DataFrame(demand[demand.Store ==1]['Sales'].astype('float'))
data.reset_index(inplace = True)
data.columns = ['ds', 'y']
#display(data)
#display(data.info())

data_train, data_test = temporal_train_test_split(data, test_size = 28)

state_holidays = pd.DataFrame(demand[demand.StateHoliday != 'None']['StateHoliday'])
state_holidays.reset_index(inplace = True)
state_holidays.columns = ['ds', 'holiday']
state_holidays['lower_window'] = 0
state_holidays['upper_window'] = 0
#display(state_holidays)

school_holidays = pd.DataFrame(demand[demand.SchoolHoliday == True]['SchoolHoliday'])
school_holidays.reset_index(inplace = True)
school_holidays.columns = ['ds', 'holiday']
school_holidays['holiday'] = 'School Holiday'
school_holidays['lower_window'] = 0
school_holidays['upper_window'] = 0
#display(school_holidays)

holidays = pd.concat((state_holidays, school_holidays))
#display(holidays)

data_train['Open'] = train['Open'].astype('int').to_numpy()
data_train['Promo'] = train['Promo'].astype('int').to_numpy()
# display(data)

p = Prophet(yearly_seasonality = True, holidays = holidays)
# p = Prophet()
p.add_regressor('Open')
p.add_regressor('Promo')
p.fit(data_train)

future = p.make_future_dataframe(periods = len(test), freq='D', include_history = False)
future['Open'] = test['Open'].to_numpy()
future['Promo'] = test['Promo'].to_numpy()
# display(future)

forecast = p.predict(future)
fig = p.plot(forecast)
fig = p.plot_components(forecast)
plt.show()

forecast.set_index('ds', inplace = True)
# display(forecast)

sns.lineplot(data = train['Sales'], label = 'Train')
sns.scatterplot(data = test['Sales'], color = 'black', label = 'Test')
sns.lineplot(data = forecast['yhat'], label = 'Prediction')
plt.title('Prophet')
plt.show()

sns.scatterplot(data = test['Sales'], color = 'black', label = 'Test')
sns.lineplot(data = forecast['yhat'], color = sns.color_palette('deep').as_hex()[1], label = 'Prediction')
plt.title('Prophet')
plt.show()

print('MAE:\t %4.2f' % (MAE(y_true = test['Sales'], y_pred = forecast['yhat'])))

#scores = cross_validation(p, initial='606 days', period='28 days', horizon = '28 days')
#metrics = performance_metrics(scores)
#display(metrics)
#print('MAE:\t %4.2f' % (metrics['mae'].mean()))

#fig = p.plot(forecast)
#fig = p.plot_components(forecast)

#### Lagged Variablen